# Gold Trading ML Model

Two approaches:
1. **Guided Model** - Trained on actual trades (mean-reversion style)
2. **Pure Model** - Learns from scratch (price prediction)

Run all cells to train and compare both models.

In [ ]:
# Setup - Clone repo if on Colab
import os

# Check if running on Colab
IN_COLAB = 'google.colab' in str(get_ipython()) if hasattr(__builtins__, '__IPYTHON__') else False

if IN_COLAB:
    # Clone or update the repo
    if os.path.exists('gold-ml-trading'):
        # Pull latest changes
        os.chdir('gold-ml-trading')
        !git pull
    else:
        !git clone https://github.com/altommo/gold-ml-trading.git
        os.chdir('gold-ml-trading')
    !pip install -r requirements.txt -q
    print("Setup complete!")
else:
    print("Running locally")

In [ ]:
# Imports and Indicator Functions (all inline to avoid caching issues)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
from xgboost import XGBClassifier
import joblib
import json
import warnings
warnings.filterwarnings('ignore')

# ===== INDICATOR FUNCTIONS =====
def calculate_wavetrend(df, n1=10, n2=21):
    df = df.copy()
    ap = (df['high'] + df['low'] + df['close']) / 3
    esa = ap.ewm(span=n1, adjust=False).mean()
    d = (ap - esa).abs().ewm(span=n1, adjust=False).mean()
    ci = (ap - esa) / (0.015 * d)
    df['wt1'] = ci.ewm(span=n2, adjust=False).mean()
    df['wt2'] = df['wt1'].rolling(4).mean()
    return df

def calculate_wolfpack(df):
    df = df.copy()
    df['wolfpack'] = df['close'].ewm(span=3, adjust=False).mean() - df['close'].ewm(span=8, adjust=False).mean()
    return df

def calculate_rsi(df, period=14):
    df = df.copy()
    delta = df['close'].diff()
    gain = delta.clip(lower=0).rolling(period).mean()
    loss = (-delta.clip(upper=0)).rolling(period).mean()
    df['rsi'] = 100 - (100 / (1 + gain / loss))
    return df

def calculate_atr(df, period=14):
    df = df.copy()
    df['atr'] = (df['high'] - df['low']).rolling(period).mean()
    df['atr_pct'] = df['atr'] / df['close'] * 100
    return df

def calculate_moving_averages(df):
    df = df.copy()
    df['ma20'] = df['close'].rolling(20).mean()
    df['ma50'] = df['close'].rolling(50).mean()
    df['ma200'] = df['close'].rolling(200).mean()
    df['price_vs_ma20'] = (df['close'] - df['ma20']) / df['ma20'] * 100
    df['price_vs_ma50'] = (df['close'] - df['ma50']) / df['ma50'] * 100
    return df

def calculate_returns(df):
    df = df.copy()
    df['ret_1h'] = df['close'].pct_change() * 100
    df['ret_4h'] = df['close'].pct_change(4) * 100
    df['ret_24h'] = df['close'].pct_change(24) * 100
    return df

def calculate_bollinger_bands(df, period=20, std_dev=2):
    df = df.copy()
    df['bb_mid'] = df['close'].rolling(period).mean()
    df['bb_std'] = df['close'].rolling(period).std()
    df['bb_upper'] = df['bb_mid'] + (df['bb_std'] * std_dev)
    df['bb_lower'] = df['bb_mid'] - (df['bb_std'] * std_dev)
    df['bb_pct'] = (df['close'] - df['bb_lower']) / (df['bb_upper'] - df['bb_lower'])
    df['bb_width'] = (df['bb_upper'] - df['bb_lower']) / df['bb_mid'] * 100
    return df

def calculate_momentum(df):
    df = df.copy()
    df['roc_5'] = (df['close'] / df['close'].shift(5) - 1) * 100
    df['roc_10'] = (df['close'] / df['close'].shift(10) - 1) * 100
    ema12 = df['close'].ewm(span=12, adjust=False).mean()
    ema26 = df['close'].ewm(span=26, adjust=False).mean()
    df['macd'] = ema12 - ema26
    df['macd_signal'] = df['macd'].ewm(span=9, adjust=False).mean()
    df['macd_hist'] = df['macd'] - df['macd_signal']
    rsi = df['rsi'] if 'rsi' in df.columns else calculate_rsi(df)['rsi']
    rsi_min = rsi.rolling(14).min()
    rsi_max = rsi.rolling(14).max()
    df['stoch_rsi'] = (rsi - rsi_min) / (rsi_max - rsi_min) * 100
    return df

def calculate_time_features(df):
    df = df.copy()
    if isinstance(df.index, pd.DatetimeIndex):
        df['hour'] = df.index.hour
        df['day_of_week'] = df.index.dayofweek
        df['is_overlap'] = ((df['hour'] >= 13) & (df['hour'] <= 17)).astype(int)
    return df

def calculate_higher_tf_trend(df, periods=[24, 48, 96]):
    df = df.copy()
    for p in periods:
        df[f'ma_{p}h'] = df['close'].rolling(p).mean()
        df[f'trend_{p}h'] = np.where(df['close'] > df[f'ma_{p}h'], 1, -1)
    df['trend_score'] = df['trend_24h'] + df['trend_48h'] + df['trend_96h']
    return df

def calculate_wt_signals(df):
    df = df.copy()
    df['wt_distance'] = abs(df['wt1'])
    return df

def add_all_indicators(df):
    df = calculate_wavetrend(df)
    df = calculate_wolfpack(df)
    df = calculate_rsi(df)
    df = calculate_atr(df)
    df = calculate_moving_averages(df)
    df = calculate_returns(df)
    df = calculate_bollinger_bands(df)
    df = calculate_momentum(df)
    df = calculate_time_features(df)
    df = calculate_higher_tf_trend(df)
    df = calculate_wt_signals(df)
    df['volatility'] = df['ret_1h'].rolling(24).std()
    df['trend'] = np.where(df['ma20'] > df['ma50'], 1, -1)
    return df

# ===== BACKTEST FUNCTION =====
def backtest_model(df, model, scaler, features, threshold=0.5, hold_hours=24):
    df = df.copy()
    df = df.dropna(subset=features)
    X = scaler.transform(df[features])
    df['prob'] = model.predict_proba(X)[:, 1]
    df['signal'] = (df['prob'] > threshold).astype(int)
    df['future_ret'] = (df['close'].shift(-hold_hours) / df['close'] - 1) * 100
    trades = df[df['signal'] == 1].copy()
    if len(trades) == 0:
        return {'total_trades': 0, 'avg_return': 0, 'total_return': 0, 'win_rate': 0, 'sharpe': 0}, pd.DataFrame()
    results = {
        'total_trades': len(trades),
        'avg_return': trades['future_ret'].mean(),
        'total_return': trades['future_ret'].sum(),
        'win_rate': (trades['future_ret'] > 0).mean() * 100,
        'sharpe': trades['future_ret'].mean() / trades['future_ret'].std() * np.sqrt(252) if trades['future_ret'].std() > 0 else 0
    }
    return results, trades

print("All functions loaded!")

## 1. Load Data

In [ ]:
# Load chart data
chart_df = pd.read_csv('data/XAUUSD_1h.csv', index_col=0, parse_dates=True)
print(f"Chart data: {len(chart_df)} bars")
print(f"Date range: {chart_df.index.min()} to {chart_df.index.max()}")

# Add indicators
chart_df = add_all_indicators(chart_df)
print("Indicators calculated")

In [ ]:
# Load trades
trades_df = pd.read_csv('data/trades_with_features.csv', parse_dates=['entry_time', 'exit_time', 'chart_time'])
print(f"Trades: {len(trades_df)}")
print(f"Winners: {len(trades_df[trades_df['won']])}, Losers: {len(trades_df[~trades_df['won']])}")
print(f"Win Rate: {trades_df['won'].mean()*100:.1f}%")

In [ ]:
# EXPANDED feature list with new indicators
FEATURES_BASIC = ['wt1', 'wt2', 'wolfpack', 'rsi', 'atr_pct', 
                  'price_vs_ma20', 'price_vs_ma50', 'ret_1h', 'ret_4h', 'ret_24h']

FEATURES_EXTENDED = FEATURES_BASIC + [
    'bb_pct', 'bb_width',           # Bollinger Bands
    'roc_5', 'roc_10',              # Rate of change
    'macd_hist', 'stoch_rsi',       # Momentum
    'hour', 'is_overlap',           # Time features
    'trend_score',                  # Higher TF trend
    'wt_distance',                  # WT distance from zero
    'volatility'                    # Volatility
]

# Use extended features
FEATURES = FEATURES_EXTENDED

# Separate buys and sells
buys = trades_df[trades_df['direction'] == 'Buy']
sells = trades_df[trades_df['direction'] == 'Sell']

print(f"Total features: {len(FEATURES)}")
print(f"Buys: {len(buys)}, Sells: {len(sells)}")

## 2. Your Trading Pattern Analysis

In [ ]:
# Your entry patterns (using basic features that exist in trades file)
print("=== YOUR BUY ENTRIES ===")
print(buys[FEATURES_BASIC].mean())

print("\n=== YOUR SELL ENTRIES ===")
print(sells[FEATURES_BASIC].mean())

In [ ]:
# Visualize your entry conditions
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# WT distribution
axes[0, 0].hist(buys['wt1'].dropna(), bins=30, alpha=0.7, label='Your Buys', color='green')
axes[0, 0].hist(sells['wt1'].dropna(), bins=30, alpha=0.7, label='Your Sells', color='red')
axes[0, 0].axvline(x=0, color='black', linestyle='--', alpha=0.5)
axes[0, 0].set_title('WaveTrend at Entry')
axes[0, 0].legend()

# RSI
axes[0, 1].hist(buys['rsi'].dropna(), bins=30, alpha=0.7, label='Your Buys', color='green')
axes[0, 1].hist(sells['rsi'].dropna(), bins=30, alpha=0.7, label='Your Sells', color='red')
axes[0, 1].axvline(x=50, color='black', linestyle='--', alpha=0.5)
axes[0, 1].set_title('RSI at Entry')
axes[0, 1].legend()

# Wolfpack
axes[1, 0].hist(buys['wolfpack'].dropna(), bins=30, alpha=0.7, label='Your Buys', color='green')
axes[1, 0].hist(sells['wolfpack'].dropna(), bins=30, alpha=0.7, label='Your Sells', color='red')
axes[1, 0].axvline(x=0, color='black', linestyle='--', alpha=0.5)
axes[1, 0].set_title('Wolfpack at Entry')
axes[1, 0].legend()

# Price vs MA20
axes[1, 1].hist(buys['price_vs_ma20'].dropna(), bins=30, alpha=0.7, label='Your Buys', color='green')
axes[1, 1].hist(sells['price_vs_ma20'].dropna(), bins=30, alpha=0.7, label='Your Sells', color='red')
axes[1, 1].axvline(x=0, color='black', linestyle='--', alpha=0.5)
axes[1, 1].set_title('Price vs MA20 % at Entry')
axes[1, 1].legend()

plt.tight_layout()
plt.show()

print("\nPattern: You BUY when indicators are LOW (dips), SELL when HIGH (rips)")
print("This is MEAN REVERSION trading.")

## 3. Create Training Labels

In [ ]:
# IMPROVED: Create labels based on YOUR mean-reversion conditions + outcome
# Instead of just marking where you traded, we label bars that:
# 1. Match your entry conditions (mean-reversion)
# 2. AND resulted in profitable moves

def create_mean_reversion_labels(df, lookahead=24, target_pct=0.5):
    """
    Label bars that match mean-reversion buy conditions and led to profit.
    Your style: Buy dips (WT negative, RSI low, price below MA)
    """
    df = df.copy()
    
    # Future return for outcome
    df['future_ret'] = (df['close'].shift(-lookahead) / df['close'] - 1) * 100
    
    # Mean-reversion BUY conditions (based on your actual trades)
    buy_conditions = (
        (df['wt1'] < 20) &          # WT not overbought
        (df['rsi'] < 60) &           # RSI not high
        (df['price_vs_ma20'] < 1.0)  # Not too far above MA20
    )
    
    # Label = conditions met AND price went up
    df['label_guided_buy'] = (buy_conditions & (df['future_ret'] > target_pct)).astype(int)
    
    # Mean-reversion SELL conditions
    sell_conditions = (
        (df['wt1'] > -20) &           # WT not oversold
        (df['rsi'] > 40) &            # RSI not low
        (df['price_vs_ma20'] > -1.0)  # Not too far below MA20
    )
    
    df['label_guided_sell'] = (sell_conditions & (df['future_ret'] < -target_pct)).astype(int)
    
    return df

# Pure model labels (simple price prediction)
LOOKAHEAD = 24
TARGET_PCT = 0.5

chart_df['future_ret'] = (chart_df['close'].shift(-LOOKAHEAD) / chart_df['close'] - 1) * 100
chart_df['label_pure_buy'] = (chart_df['future_ret'] > TARGET_PCT).astype(int)
chart_df['label_pure_sell'] = (chart_df['future_ret'] < -TARGET_PCT).astype(int)

# Guided labels with mean-reversion conditions
chart_df = create_mean_reversion_labels(chart_df, lookahead=LOOKAHEAD, target_pct=TARGET_PCT)

print(f"Guided buy labels: {chart_df['label_guided_buy'].sum()} ({chart_df['label_guided_buy'].mean()*100:.1f}%)")
print(f"Pure buy labels: {chart_df['label_pure_buy'].sum()} ({chart_df['label_pure_buy'].mean()*100:.1f}%)")
print(f"\nGuided model now learns: 'When conditions match your style, did price go up?'")

## 4. Train Guided Model (Your Style)

In [ ]:
# Prepare training data with EXTENDED features
df_train = chart_df.dropna(subset=FEATURES + ['label_guided_buy'])
X = df_train[FEATURES]
y = df_train['label_guided_buy']

print(f"Training samples: {len(X)}")
print(f"Positive samples: {y.sum()} ({y.mean()*100:.2f}%)")

# Split - use time-based split for financial data
split_idx = int(len(X) * 0.8)
X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]

print(f"Train: {len(X_train)}, Test: {len(X_test)}")

# Scale
scaler_guided = StandardScaler()
X_train_scaled = scaler_guided.fit_transform(X_train)
X_test_scaled = scaler_guided.transform(X_test)

In [ ]:
# Train Guided Model with FIXED PARAMS or HYPERPARAMETER TUNING
# Set USE_FIXED_PARAMS = True to reproduce v0.1, False for hyperparameter search
USE_FIXED_PARAMS = True

# Parameter grid for search (if USE_FIXED_PARAMS = False)
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7, 10],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'min_child_weight': [1, 3, 5],
    'subsample': [0.7, 0.8, 0.9],
    'colsample_bytree': [0.7, 0.8, 0.9],
    'gamma': [0, 0.1, 0.2]
}

if USE_FIXED_PARAMS:
    print("Using FIXED params (v0.1 config)...")
    guided_model = XGBClassifier(
        n_estimators=200,
        max_depth=3,
        learning_rate=0.2,
        min_child_weight=3,
        subsample=0.8,
        colsample_bytree=0.7,
        gamma=0.1,
        random_state=42,
        eval_metric='logloss',
        use_label_encoder=False
    )
    guided_model.fit(X_train_scaled, y_train)
    search = type('obj', (object,), {'best_params_': guided_model.get_params()})()
else:
    print("Tuning Guided Model hyperparameters...")
    search = RandomizedSearchCV(
        XGBClassifier(random_state=42, eval_metric='logloss', use_label_encoder=False),
        param_grid, 
        n_iter=30,
        cv=3,
        scoring='f1',
        random_state=42,
        n_jobs=-1
    )
    search.fit(X_train_scaled, y_train)
    guided_model = search.best_estimator_
    print(f"Best params: {search.best_params_}")

# Evaluate on test set
y_pred = guided_model.predict(X_test_scaled)
y_prob = guided_model.predict_proba(X_test_scaled)[:, 1]

print("\n=== GUIDED MODEL RESULTS ===")
print(classification_report(y_test, y_pred))

In [ ]:
# Feature importance - Guided (top 15)
importance_guided = pd.DataFrame({
    'feature': FEATURES,
    'importance': guided_model.feature_importances_
}).sort_values('importance', ascending=True).tail(15)

plt.figure(figsize=(10, 8))
plt.barh(importance_guided['feature'], importance_guided['importance'], color='steelblue')
plt.title('Guided Model - Top 15 Features for YOUR Entry Style')
plt.xlabel('Importance')
plt.tight_layout()
plt.show()

print(f"\nTop 5 features for Guided model:")
for _, row in importance_guided.tail(5).iloc[::-1].iterrows():
    print(f"  {row['feature']}: {row['importance']:.4f}")

## 5. Train Pure Model (From Scratch)

In [ ]:
# Pure model training data with EXTENDED features
df_pure = chart_df.dropna(subset=FEATURES + ['label_pure_buy'])
X_pure = df_pure[FEATURES]
y_pure = df_pure['label_pure_buy']

print(f"Training samples: {len(X_pure)}")
print(f"Positive samples: {y_pure.sum()} ({y_pure.mean()*100:.1f}%)")

# Time-based split
split_idx = int(len(X_pure) * 0.8)
X_train_p, X_test_p = X_pure.iloc[:split_idx], X_pure.iloc[split_idx:]
y_train_p, y_test_p = y_pure.iloc[:split_idx], y_pure.iloc[split_idx:]

print(f"Train: {len(X_train_p)}, Test: {len(X_test_p)}")

# Scale
scaler_pure = StandardScaler()
X_train_p_scaled = scaler_pure.fit_transform(X_train_p)
X_test_p_scaled = scaler_pure.transform(X_test_p)

In [ ]:
# Train Pure Model with FIXED PARAMS (v0.1 - 125% return config)
# Set USE_FIXED_PARAMS = True to reproduce v0.1, False for hyperparameter search
USE_FIXED_PARAMS = True

if USE_FIXED_PARAMS:
    print("Using FIXED params (v0.1 config)...")
    pure_model = XGBClassifier(
        n_estimators=300,
        max_depth=10,
        learning_rate=0.2,
        min_child_weight=3,
        subsample=0.7,
        colsample_bytree=0.9,
        gamma=0.1,
        random_state=42,
        eval_metric='logloss',
        use_label_encoder=False
    )
    pure_model.fit(X_train_p_scaled, y_train_p)
    search_pure = type('obj', (object,), {'best_params_': pure_model.get_params()})()
else:
    print("Tuning Pure Model hyperparameters...")
    search_pure = RandomizedSearchCV(
        XGBClassifier(random_state=42, eval_metric='logloss', use_label_encoder=False),
        param_grid, 
        n_iter=30,
        cv=3,
        scoring='f1',
        random_state=42,
        n_jobs=-1
    )
    search_pure.fit(X_train_p_scaled, y_train_p)
    pure_model = search_pure.best_estimator_
    print(f"Best params: {search_pure.best_params_}")

y_pred_p = pure_model.predict(X_test_p_scaled)
y_prob_p = pure_model.predict_proba(X_test_p_scaled)[:, 1]

print("\n=== PURE MODEL RESULTS ===")
print(classification_report(y_test_p, y_pred_p))

In [ ]:
# Feature importance - Pure (top 15)
importance_pure = pd.DataFrame({
    'feature': FEATURES,
    'importance': pure_model.feature_importances_
}).sort_values('importance', ascending=True).tail(15)

plt.figure(figsize=(10, 8))
plt.barh(importance_pure['feature'], importance_pure['importance'], color='darkorange')
plt.title('Pure Model - Top 15 Features for Price Prediction')
plt.xlabel('Importance')
plt.tight_layout()
plt.show()

print(f"\nTop 5 features for Pure model:")
for _, row in importance_pure.tail(5).iloc[::-1].iterrows():
    print(f"  {row['feature']}: {row['importance']:.4f}")

## 6. Compare Feature Importance

In [ ]:
# Side by side comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

axes[0].barh(importance_guided['feature'], importance_guided['importance'], color='blue')
axes[0].set_title('GUIDED: Your Trading Style')
axes[0].set_xlabel('Importance')

axes[1].barh(importance_pure['feature'], importance_pure['importance'], color='orange')
axes[1].set_title('PURE: Price Prediction')
axes[1].set_xlabel('Importance')

plt.tight_layout()
plt.show()

print("\nDifference shows what YOU focus on vs what statistically predicts price movement")

## 7. Backtest Both Models

In [ ]:
# Backtest on last 500 bars (out of sample)
test_data = chart_df.tail(500).copy()
print(f"Backtest period: {test_data.index.min()} to {test_data.index.max()}")

# ===== THRESHOLD SETTINGS =====
# Set to None for auto-optimization, or set fixed value (e.g. 0.5)
FORCE_GUIDED_THRESHOLD = None  # e.g. 0.3
FORCE_PURE_THRESHOLD = 0.5     # v0.1 used 0.5 for 125% return
# ==============================

# Find optimal thresholds
def optimize_threshold(df, model, scaler, features, thresholds=[0.3, 0.4, 0.5, 0.6, 0.7]):
    """Test different thresholds and find best one"""
    results = []
    for thresh in thresholds:
        res, _ = backtest_model(df, model, scaler, features, threshold=thresh, hold_hours=24)
        res['threshold'] = thresh
        results.append(res)
    return pd.DataFrame(results)

print("\n=== THRESHOLD OPTIMIZATION ===")
thresholds = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]

guided_thresh_results = optimize_threshold(test_data, guided_model, scaler_guided, FEATURES, thresholds)
pure_thresh_results = optimize_threshold(test_data, pure_model, scaler_pure, FEATURES, thresholds)

# Show all threshold results
print("\nGuided threshold sweep:")
print(guided_thresh_results[['threshold', 'total_trades', 'win_rate', 'total_return', 'sharpe']].to_string(index=False))

print("\nPure threshold sweep:")
print(pure_thresh_results[['threshold', 'total_trades', 'win_rate', 'total_return', 'sharpe']].to_string(index=False))

# Use forced or optimized threshold
if FORCE_GUIDED_THRESHOLD is not None:
    best_guided_thresh = FORCE_GUIDED_THRESHOLD
    print(f"\nUsing FORCED Guided threshold: {best_guided_thresh}")
else:
    best_guided_thresh = guided_thresh_results.loc[guided_thresh_results['sharpe'].idxmax(), 'threshold']
    print(f"\nAuto-selected Guided threshold: {best_guided_thresh}")

if FORCE_PURE_THRESHOLD is not None:
    best_pure_thresh = FORCE_PURE_THRESHOLD
    print(f"Using FORCED Pure threshold: {best_pure_thresh}")
else:
    best_pure_thresh = pure_thresh_results.loc[pure_thresh_results['sharpe'].idxmax(), 'threshold']
    print(f"Auto-selected Pure threshold: {best_pure_thresh}")

# Run with selected thresholds
guided_results, guided_trades = backtest_model(
    test_data, guided_model, scaler_guided, FEATURES, threshold=best_guided_thresh, hold_hours=24
)
pure_results, pure_trades = backtest_model(
    test_data, pure_model, scaler_pure, FEATURES, threshold=best_pure_thresh, hold_hours=24
)

print("\n=== BACKTEST COMPARISON ===")
comparison = pd.DataFrame({
    'Metric': ['Threshold', 'Trades', 'Win Rate %', 'Avg Return %', 'Total Return %', 'Sharpe'],
    'Guided (Your Style)': [
        f"{best_guided_thresh}",
        guided_results['total_trades'],
        f"{guided_results['win_rate']:.1f}",
        f"{guided_results['avg_return']:.2f}",
        f"{guided_results['total_return']:.2f}",
        f"{guided_results['sharpe']:.2f}"
    ],
    'Pure (From Scratch)': [
        f"{best_pure_thresh}",
        pure_results['total_trades'],
        f"{pure_results['win_rate']:.1f}",
        f"{pure_results['avg_return']:.2f}",
        f"{pure_results['total_return']:.2f}",
        f"{pure_results['sharpe']:.2f}"
    ]
})
print(comparison.to_string(index=False))

## 8. Ensemble Model (Best of Both)

In [ ]:
# ENSEMBLE: Combine both models
# Only take trades when BOTH models agree

def backtest_ensemble(df, guided_model, pure_model, scaler_g, scaler_p, features, 
                      g_thresh=0.5, p_thresh=0.5, hold_hours=24):
    """Backtest ensemble - only trade when both models agree"""
    df = df.copy()
    df = df.dropna(subset=features)
    
    X_g = scaler_g.transform(df[features])
    X_p = scaler_p.transform(df[features])
    
    df['prob_guided'] = guided_model.predict_proba(X_g)[:, 1]
    df['prob_pure'] = pure_model.predict_proba(X_p)[:, 1]
    
    # Ensemble: both must agree
    df['signal'] = ((df['prob_guided'] > g_thresh) & (df['prob_pure'] > p_thresh)).astype(int)
    
    # Calculate future returns
    df['future_ret'] = (df['close'].shift(-hold_hours) / df['close'] - 1) * 100
    
    trades = df[df['signal'] == 1].copy()
    
    if len(trades) == 0:
        return {'total_trades': 0, 'avg_return': 0, 'total_return': 0, 
                'win_rate': 0, 'sharpe': 0, 'max_drawdown': 0}, pd.DataFrame()
    
    results = {
        'total_trades': len(trades),
        'avg_return': trades['future_ret'].mean(),
        'total_return': trades['future_ret'].sum(),
        'win_rate': (trades['future_ret'] > 0).mean() * 100,
        'sharpe': trades['future_ret'].mean() / trades['future_ret'].std() * np.sqrt(252) if trades['future_ret'].std() > 0 else 0,
        'max_drawdown': 0  # simplified
    }
    
    return results, trades

# Test ensemble with various threshold combinations
print("=== ENSEMBLE MODEL (Both Must Agree) ===\n")

ensemble_results = []
for g_t in [0.3, 0.4, 0.5]:
    for p_t in [0.4, 0.5, 0.6]:
        res, _ = backtest_ensemble(test_data, guided_model, pure_model, 
                                   scaler_guided, scaler_pure, FEATURES,
                                   g_thresh=g_t, p_thresh=p_t)
        res['g_thresh'] = g_t
        res['p_thresh'] = p_t
        ensemble_results.append(res)

ensemble_df = pd.DataFrame(ensemble_results)
ensemble_df = ensemble_df.sort_values('sharpe', ascending=False)

print("Top 5 Ensemble Configurations:")
print(ensemble_df.head().to_string(index=False))

# Best ensemble
best_g = ensemble_df.iloc[0]['g_thresh']
best_p = ensemble_df.iloc[0]['p_thresh']
ensemble_final, ensemble_trades = backtest_ensemble(
    test_data, guided_model, pure_model, scaler_guided, scaler_pure, FEATURES,
    g_thresh=best_g, p_thresh=best_p
)

print(f"\n=== BEST ENSEMBLE (G={best_g}, P={best_p}) ===")
print(f"Trades: {ensemble_final['total_trades']}")
print(f"Win Rate: {ensemble_final['win_rate']:.1f}%")
print(f"Avg Return: {ensemble_final['avg_return']:.2f}%")
print(f"Total Return: {ensemble_final['total_return']:.2f}%")
print(f"Sharpe: {ensemble_final['sharpe']:.2f}")

In [ ]:
# Get current signal from latest bar
latest = chart_df.dropna(subset=FEATURES).tail(1)

if len(latest) > 0:
    X_latest_g = scaler_guided.transform(latest[FEATURES])
    X_latest_p = scaler_pure.transform(latest[FEATURES])
    
    guided_prob = guided_model.predict_proba(X_latest_g)[0, 1]
    pure_prob = pure_model.predict_proba(X_latest_p)[0, 1]
    
    print("=" * 50)
    print("CURRENT SIGNAL")
    print("=" * 50)
    print(f"Time: {latest.index[0]}")
    print(f"Price: ${latest['close'].values[0]:.2f}")
    print(f"\nIndicators:")
    print(f"  WaveTrend: {latest['wt1'].values[0]:.1f}")
    print(f"  Wolfpack: {latest['wolfpack'].values[0]:.2f}")
    print(f"  RSI: {latest['rsi'].values[0]:.1f}")
    print(f"  Price vs MA20: {latest['price_vs_ma20'].values[0]:.2f}%")
    print(f"  Trend Score: {latest['trend_score'].values[0]:.0f}")
    print(f"\nBUY Probability:")
    print(f"  Guided (Your Style): {guided_prob*100:.1f}%")
    print(f"  Pure (ML Optimal): {pure_prob*100:.1f}%")
    
    # Ensemble signal
    ensemble_signal = (guided_prob > best_g) and (pure_prob > best_p)
    print(f"\n  ENSEMBLE SIGNAL: {'BUY' if ensemble_signal else 'NO TRADE'}")
    
    print(f"\nInterpretation:")
    if guided_prob > best_g:
        print(f"  Guided: BULLISH - This matches your entry style")
    else:
        print(f"  Guided: NEUTRAL - Not your typical setup")
    
    if pure_prob > best_p:
        print(f"  Pure: BULLISH - ML predicts price going up")
    else:
        print(f"  Pure: NEUTRAL - No strong upside signal")

## 10. Save Models

In [ ]:
# Save models and configuration
import os
import json

os.makedirs('models', exist_ok=True)

# Save models
joblib.dump(guided_model, 'models/guided_model.pkl')
joblib.dump(pure_model, 'models/pure_model.pkl')
joblib.dump(scaler_guided, 'models/scaler_guided.pkl')
joblib.dump(scaler_pure, 'models/scaler_pure.pkl')

# Save configuration
config = {
    'features': FEATURES,
    'best_guided_threshold': float(best_guided_thresh),
    'best_pure_threshold': float(best_pure_thresh),
    'ensemble_guided_threshold': float(best_g),
    'ensemble_pure_threshold': float(best_p),
    'lookahead_hours': LOOKAHEAD,
    'target_pct': TARGET_PCT,
    'guided_params': search.best_params_,
    'pure_params': search_pure.best_params_
}

with open('models/config.json', 'w') as f:
    json.dump(config, f, indent=2, default=str)

print("Saved:")
print("  - models/guided_model.pkl")
print("  - models/pure_model.pkl")
print("  - models/scaler_guided.pkl")
print("  - models/scaler_pure.pkl")
print("  - models/config.json")

# Download if on Colab
if IN_COLAB:
    from google.colab import files
    import shutil
    shutil.make_archive('models', 'zip', 'models')
    files.download('models.zip')
    print("\nDownloaded models.zip")

In [ ]:
# Save all results to file
import os
from datetime import datetime

os.makedirs('results', exist_ok=True)
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

# Compile all results
results_summary = {
    'timestamp': timestamp,
    'backtest_period': {
        'start': str(test_data.index.min()),
        'end': str(test_data.index.max())
    },
    'features_used': FEATURES,
    'lookahead_hours': LOOKAHEAD,
    'target_pct': TARGET_PCT,
    
    'guided_model': {
        'best_threshold': float(best_guided_thresh),
        'trades': int(guided_results['total_trades']),
        'win_rate': float(guided_results['win_rate']),
        'avg_return': float(guided_results['avg_return']),
        'total_return': float(guided_results['total_return']),
        'sharpe': float(guided_results['sharpe']),
        'best_params': {k: str(v) for k, v in search.best_params_.items()}
    },
    
    'pure_model': {
        'best_threshold': float(best_pure_thresh),
        'trades': int(pure_results['total_trades']),
        'win_rate': float(pure_results['win_rate']),
        'avg_return': float(pure_results['avg_return']),
        'total_return': float(pure_results['total_return']),
        'sharpe': float(pure_results['sharpe']),
        'best_params': {k: str(v) for k, v in search_pure.best_params_.items()}
    },
    
    'ensemble': {
        'guided_threshold': float(best_g),
        'pure_threshold': float(best_p),
        'trades': int(ensemble_final['total_trades']),
        'win_rate': float(ensemble_final['win_rate']),
        'avg_return': float(ensemble_final['avg_return']),
        'total_return': float(ensemble_final['total_return']),
        'sharpe': float(ensemble_final['sharpe'])
    },
    
    'all_ensemble_configs': ensemble_df.to_dict('records'),
    'guided_threshold_sweep': guided_thresh_results.to_dict('records'),
    'pure_threshold_sweep': pure_thresh_results.to_dict('records')
}

# Save JSON
results_file = f'results/backtest_{timestamp}.json'
with open(results_file, 'w') as f:
    json.dump(results_summary, f, indent=2, default=str)

# Save trades to CSV
if len(guided_trades) > 0:
    guided_trades.to_csv(f'results/guided_trades_{timestamp}.csv')
if len(pure_trades) > 0:
    pure_trades.to_csv(f'results/pure_trades_{timestamp}.csv')
if len(ensemble_trades) > 0:
    ensemble_trades.to_csv(f'results/ensemble_trades_{timestamp}.csv')

print(f"Results saved to: {results_file}")
print(f"\n{'='*60}")
print("SUMMARY")
print('='*60)
print(f"\nBacktest: {results_summary['backtest_period']['start']} to {results_summary['backtest_period']['end']}")
print(f"\n{'Model':<12} {'Trades':>8} {'Win%':>8} {'Return%':>10} {'Sharpe':>8}")
print('-'*50)
print(f"{'Guided':<12} {guided_results['total_trades']:>8} {guided_results['win_rate']:>7.1f}% {guided_results['total_return']:>9.2f}% {guided_results['sharpe']:>8.2f}")
print(f"{'Pure':<12} {pure_results['total_trades']:>8} {pure_results['win_rate']:>7.1f}% {pure_results['total_return']:>9.2f}% {pure_results['sharpe']:>8.2f}")
print(f"{'Ensemble':<12} {ensemble_final['total_trades']:>8} {ensemble_final['win_rate']:>7.1f}% {ensemble_final['total_return']:>9.2f}% {ensemble_final['sharpe']:>8.2f}")

# Download if on Colab
if IN_COLAB:
    from google.colab import files
    files.download(results_file)